In [177]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
import random
import numpy as np
import pandas as pd

import argo_utils.argo_utils as argo_utils
import math

In [179]:
def return_random_num(y, fraud_min, fraud_max, nonfraud_min, nonfraud_max, rand_func):
    data = [rand_func(fraud_min, fraud_max) if i == 1 else rand_func(
        nonfraud_min, nonfraud_max) for i in y]
    return data

random.seed(0)
np.random.seed(0)
y = pd.Series(data=[0]*980 + [1]*20, index=list(range(0, 1000)))
X = pd.DataFrame(data={
    "num_distinct_txn_per_email_1day": [round(max(i, 0)) for i in return_random_num(y, 2, 1, 1, 2, np.random.normal)],
    "num_distinct_txn_per_email_7day": [round(max(i, 0)) for i in return_random_num(y, 4, 2, 2, 3, np.random.normal)],
    "ip_country_us": [round(min(i, 1)) for i in [max(i, 0) for i in return_random_num(y, 0.3, 0.4, 0.5, 0.5, np.random.normal)]],
    "email_kb_distance": [min(i, 1) for i in [max(i, 0) for i in return_random_num(y, 0.2, 0.5, 0.6, 0.4, np.random.normal)]],
    "email_alpharatio":  [min(i, 1) for i in [max(i, 0) for i in return_random_num(y, 0.33, 0.1, 0.5, 0.2, np.random.normal)]],
},
    index=list(range(0, 1000))
)
columns_int = [
    'num_distinct_txn_per_email_1day', 'num_distinct_txn_per_email_7day', 'ip_country_us']
columns_cat = ['ip_country_us']
columns_num = ['num_distinct_txn_per_email_1day',
               'num_distinct_txn_per_email_7day', 'email_kb_distance', 'email_alpharatio']
weights = y.apply(lambda x: 1000 if x == 1 else 1)

In [180]:
X.describe()

,num_distinct_txn_per_email_1day,num_distinct_txn_per_email_7day,ip_country_us,email_kb_distance,email_alpharatio
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.340000,2.491000,0.471000,0.563383,0.501840
std,1.494869,2.354881,0.499408,0.326518,0.197752
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.306039,0.366063
50%,1.000000,2.000000,0.000000,0.585855,0.499346
75%,2.000000,4.000000,1.000000,0.862649,0.638693
max,7.000000,12.000000,1.000000,1.000000,1.000000


In [192]:
from rule_generation.rule_generator_opt import RuleGeneratorOpt
from rule_generation.rule_generator_dt import RuleGeneratorDT
from sklearn.ensemble import RandomForestClassifier
# from rule_generation.rule_generator_opt_old import RuleGeneratorOpt as RuleGeneratorOptOld
from rule_optimisation.optimisation_functions import FScore

In [193]:
f = FScore(0.5)

In [198]:
# rg = RuleGeneratorOpt(opt_func=f.fit, n_total_conditions=4, num_rules_keep=50, n_points= 10, ratio_window= 2, 
#                       remove_corr_rules= False)
rg = RuleGeneratorDT(opt_func=f.fit, n_total_conditions=4, tree_ensemble=RandomForestClassifier(random_state=9), num_cores=1)

In [199]:
X_rules = rg.fit(X, y)

In [200]:
rg.rule_descriptions

,Logic,Precision,Recall,nConditions,PercDataFlagged,OptMetric
Rule,,,,,,
ARGORule50,(X['email_alpharatio']<=0.37224)&(X['email_alp...,0.714286,0.25,4,0.007,0.520833
ARGORule381,(X['email_alpharatio']<=0.5251)&(X['email_kb_d...,0.545455,0.30,4,0.011,0.468750
ARGORule425,(X['email_alpharatio']<=0.5251)&(X['email_kb_d...,0.545455,0.30,4,0.011,0.468750
ARGORule151,(X['email_alpharatio']<=0.35805)&(X['email_alp...,0.666667,0.20,4,0.006,0.454545
ARGORule75,(X['email_alpharatio']<=0.37273)&(X['email_kb_...,0.666667,0.20,4,0.006,0.454545
...,...,...,...,...,...,...
ARGORule194,(X['email_alpharatio']>0.43844)&(X['email_kb_d...,0.009174,0.05,3,0.109,0.010965
ARGORule469,(X['email_alpharatio']>0.37356)&(X['email_kb_d...,0.008621,0.15,3,0.348,0.010623
ARGORule100,(X['email_alpharatio']<=0.45402)&(X['email_alp...,0.008772,0.05,4,0.114,0.010504


In [170]:
rg.rule_descriptions.index.duplicated().sum()

0

In [160]:
X_rules.columns.duplicated().sum()

0

In [80]:
pairwise_to_orig_lookup = {"(X['email_alpharatio']<=0.33)&(X['email_kb_distance']<=0.0)": ['ARGORule12', 'ARGORule14', 'ARGORule20', ["(X['email_kb_distance']<=0.0)", "(X['email_alpharatio']<=0.33)"]], "(X['email_kb_distance']<=0.0)&(X['num_distinct_txn_per_email_1day']>=2)": ['ARGORule12', 'ARGORule0', 'ARGORule21', ["(X['email_kb_distance']<=0.0)", "(X['num_distinct_txn_per_email_1day']>=2)"]], "(X['email_kb_distance']<=0.0)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule12', 'ARGORule2', 'ARGORule22', ["(X['email_kb_distance']<=0.0)", "(X['num_distinct_txn_per_email_7day']>=7)"]], "(X['email_kb_distance']<=0.0)&(X['ip_country_us']==False)": ['ARGORule12', 'ARGORule16', 'ARGORule23', ["(X['email_kb_distance']<=0.0)", "(X['ip_country_us']==False)"]], "(X['email_kb_distance']<=0.0)&(X['num_distinct_txn_per_email_1day']<=3)": ['ARGORule12', 'ARGORule8', 'ARGORule24', ["(X['email_kb_distance']<=0.0)", "(X['num_distinct_txn_per_email_1day']<=3)"]], "(X['email_kb_distance']<=0.0)&(X['num_distinct_txn_per_email_7day']<=4)": ['ARGORule12', 'ARGORule10', 'ARGORule25', ["(X['email_kb_distance']<=0.0)", "(X['num_distinct_txn_per_email_7day']<=4)"]], "(X['email_kb_distance']<=0.0)&(X['email_kb_distance']>=0.94)": ['ARGORule12', 'ARGORule4', 'ARGORule26', ["(X['email_kb_distance']<=0.0)", "(X['email_kb_distance']>=0.94)"]], "(X['email_alpharatio']>=0.5)&(X['email_kb_distance']<=0.0)": ['ARGORule12', 'ARGORule6', 'ARGORule27', ["(X['email_kb_distance']<=0.0)", "(X['email_alpharatio']>=0.5)"]], "(X['email_alpharatio']<=0.33)&(X['num_distinct_txn_per_email_1day']>=2)": ['ARGORule14', 'ARGORule0', 'ARGORule28', ["(X['email_alpharatio']<=0.33)", "(X['num_distinct_txn_per_email_1day']>=2)"]], "(X['email_alpharatio']<=0.33)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule14', 'ARGORule2', 'ARGORule29', ["(X['email_alpharatio']<=0.33)", "(X['num_distinct_txn_per_email_7day']>=7)"]], "(X['email_alpharatio']<=0.33)&(X['ip_country_us']==False)": ['ARGORule14', 'ARGORule16', 'ARGORule30', ["(X['email_alpharatio']<=0.33)", "(X['ip_country_us']==False)"]], "(X['email_alpharatio']<=0.33)&(X['num_distinct_txn_per_email_1day']<=3)": ['ARGORule14', 'ARGORule8', 'ARGORule31', ["(X['email_alpharatio']<=0.33)", "(X['num_distinct_txn_per_email_1day']<=3)"]], "(X['email_alpharatio']<=0.33)&(X['num_distinct_txn_per_email_7day']<=4)": ['ARGORule14', 'ARGORule10', 'ARGORule32', ["(X['email_alpharatio']<=0.33)", "(X['num_distinct_txn_per_email_7day']<=4)"]], "(X['email_alpharatio']<=0.33)&(X['email_kb_distance']>=0.94)": ['ARGORule14', 'ARGORule4', 'ARGORule33', ["(X['email_alpharatio']<=0.33)", "(X['email_kb_distance']>=0.94)"]], "(X['email_alpharatio']<=0.33)&(X['email_alpharatio']>=0.5)": ['ARGORule14', 'ARGORule6', 'ARGORule34', ["(X['email_alpharatio']<=0.33)", "(X['email_alpharatio']>=0.5)"]], "(X['num_distinct_txn_per_email_1day']>=2)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule0', 'ARGORule2', 'ARGORule35', ["(X['num_distinct_txn_per_email_1day']>=2)", "(X['num_distinct_txn_per_email_7day']>=7)"]], "(X['ip_country_us']==False)&(X['num_distinct_txn_per_email_1day']>=2)": ['ARGORule0', 'ARGORule16', 'ARGORule36', ["(X['num_distinct_txn_per_email_1day']>=2)", "(X['ip_country_us']==False)"]], "(X['num_distinct_txn_per_email_1day']<=3)&(X['num_distinct_txn_per_email_1day']>=2)": ['ARGORule0', 'ARGORule8', 'ARGORule37', ["(X['num_distinct_txn_per_email_1day']>=2)", "(X['num_distinct_txn_per_email_1day']<=3)"]], "(X['num_distinct_txn_per_email_1day']>=2)&(X['num_distinct_txn_per_email_7day']<=4)": ['ARGORule0', 'ARGORule10', 'ARGORule38', ["(X['num_distinct_txn_per_email_1day']>=2)", "(X['num_distinct_txn_per_email_7day']<=4)"]], "(X['email_kb_distance']>=0.94)&(X['num_distinct_txn_per_email_1day']>=2)": ['ARGORule0', 'ARGORule4', 'ARGORule39', ["(X['num_distinct_txn_per_email_1day']>=2)", "(X['email_kb_distance']>=0.94)"]], "(X['email_alpharatio']>=0.5)&(X['num_distinct_txn_per_email_1day']>=2)": ['ARGORule0', 'ARGORule6', 'ARGORule40', ["(X['num_distinct_txn_per_email_1day']>=2)", "(X['email_alpharatio']>=0.5)"]], "(X['ip_country_us']==False)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule2', 'ARGORule16', 'ARGORule41', ["(X['num_distinct_txn_per_email_7day']>=7)", "(X['ip_country_us']==False)"]], "(X['num_distinct_txn_per_email_1day']<=3)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule2', 'ARGORule8', 'ARGORule42', ["(X['num_distinct_txn_per_email_7day']>=7)", "(X['num_distinct_txn_per_email_1day']<=3)"]], "(X['num_distinct_txn_per_email_7day']<=4)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule2', 'ARGORule10', 'ARGORule43', ["(X['num_distinct_txn_per_email_7day']>=7)", "(X['num_distinct_txn_per_email_7day']<=4)"]], "(X['email_kb_distance']>=0.94)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule2', 'ARGORule4', 'ARGORule44', ["(X['num_distinct_txn_per_email_7day']>=7)", "(X['email_kb_distance']>=0.94)"]], "(X['email_alpharatio']>=0.5)&(X['num_distinct_txn_per_email_7day']>=7)": ['ARGORule2', 'ARGORule6', 'ARGORule45', ["(X['num_distinct_txn_per_email_7day']>=7)", "(X['email_alpharatio']>=0.5)"]], "(X['ip_country_us']==False)&(X['num_distinct_txn_per_email_1day']<=3)": ['ARGORule16', 'ARGORule8', 'ARGORule46', ["(X['ip_country_us']==False)", "(X['num_distinct_txn_per_email_1day']<=3)"]], "(X['ip_country_us']==False)&(X['num_distinct_txn_per_email_7day']<=4)": ['ARGORule16', 'ARGORule10', 'ARGORule47', ["(X['ip_country_us']==False)", "(X['num_distinct_txn_per_email_7day']<=4)"]], "(X['email_kb_distance']>=0.94)&(X['ip_country_us']==False)": ['ARGORule16', 'ARGORule4', 'ARGORule48', ["(X['ip_country_us']==False)", "(X['email_kb_distance']>=0.94)"]], "(X['email_alpharatio']>=0.5)&(X['ip_country_us']==False)": ['ARGORule16', 'ARGORule6', 'ARGORule49', ["(X['ip_country_us']==False)", "(X['email_alpharatio']>=0.5)"]], "(X['num_distinct_txn_per_email_1day']<=3)&(X['num_distinct_txn_per_email_7day']<=4)": ['ARGORule8', 'ARGORule10', 'ARGORule50', ["(X['num_distinct_txn_per_email_1day']<=3)", "(X['num_distinct_txn_per_email_7day']<=4)"]], "(X['email_kb_distance']>=0.94)&(X['num_distinct_txn_per_email_1day']<=3)": ['ARGORule8', 'ARGORule4', 'ARGORule51', ["(X['num_distinct_txn_per_email_1day']<=3)", "(X['email_kb_distance']>=0.94)"]], "(X['email_alpharatio']>=0.5)&(X['num_distinct_txn_per_email_1day']<=3)": ['ARGORule8', 'ARGORule6', 'ARGORule52', ["(X['num_distinct_txn_per_email_1day']<=3)", "(X['email_alpharatio']>=0.5)"]], "(X['email_kb_distance']>=0.94)&(X['num_distinct_txn_per_email_7day']<=4)": ['ARGORule10', 'ARGORule4', 'ARGORule53', ["(X['num_distinct_txn_per_email_7day']<=4)", "(X['email_kb_distance']>=0.94)"]], "(X['email_alpharatio']>=0.5)&(X['num_distinct_txn_per_email_7day']<=4)": ['ARGORule10', 'ARGORule6', 'ARGORule54', ["(X['num_distinct_txn_per_email_7day']<=4)", "(X['email_alpharatio']>=0.5)"]], "(X['email_alpharatio']>=0.5)&(X['email_kb_distance']>=0.94)": ['ARGORule4', 'ARGORule6', 'ARGORule55', ["(X['email_kb_distance']>=0.94)", "(X['email_alpharatio']>=0.5)"]]}

In [84]:
dict((pairwise_info_list[2], pairwise_info_list[3]) for pairwise_rule_logic, pairwise_info_list in pairwise_to_orig_lookup.items())

{'ARGORule20': ["(X['email_kb_distance']<=0.0)",
  "(X['email_alpharatio']<=0.33)"],
 'ARGORule21': ["(X['email_kb_distance']<=0.0)",
  "(X['num_distinct_txn_per_email_1day']>=2)"],
 'ARGORule22': ["(X['email_kb_distance']<=0.0)",
  "(X['num_distinct_txn_per_email_7day']>=7)"],
 'ARGORule23': ["(X['email_kb_distance']<=0.0)",
  "(X['ip_country_us']==False)"],
 'ARGORule24': ["(X['email_kb_distance']<=0.0)",
  "(X['num_distinct_txn_per_email_1day']<=3)"],
 'ARGORule25': ["(X['email_kb_distance']<=0.0)",
  "(X['num_distinct_txn_per_email_7day']<=4)"],
 'ARGORule26': ["(X['email_kb_distance']<=0.0)",
  "(X['email_kb_distance']>=0.94)"],
 'ARGORule27': ["(X['email_kb_distance']<=0.0)",
  "(X['email_alpharatio']>=0.5)"],
 'ARGORule28': ["(X['email_alpharatio']<=0.33)",
  "(X['num_distinct_txn_per_email_1day']>=2)"],
 'ARGORule29': ["(X['email_alpharatio']<=0.33)",
  "(X['num_distinct_txn_per_email_7day']>=7)"],
 'ARGORule30': ["(X['email_alpharatio']<=0.33)",
  "(X['ip_country_us']==False)"

In [78]:
rg.rule_descriptions.loc[['ARGORule30', 'ARGORule32']]['Logic'].values

KeyError: "None of [Index(['ARGORule30', 'ARGORule32'], dtype='object', name='Rule')] are in the [index]"

In [19]:
# self.rule_descriptions

In [20]:
# self.apply(X, y)

In [21]:
# self.rule_descriptions_applied

In [22]:
# print(self.rule_descriptions.shape == self.rule_descriptions_applied.shape)
# print(self.rule_descriptions.columns == self.rule_descriptions_applied.columns)
# print((self.rule_descriptions.index == self.rule_descriptions_applied.index).mean())

In [23]:
# (self.rule_descriptions.sort_index() == self.rule_descriptions_applied.sort_index()).mean()

In [57]:
rg_old = RuleGeneratorOptOld(opt_func=f.fit, n_total_conditions=4, num_rules_keep=50, n_points= 10, ratio_window= 2, 
                      remove_corr_rules= False)

In [58]:
rg_old.fit(X, y)

(8, 6)
(9, 6)
9
Pairwise shape, n_loop = 1: (36, 6)
Pairwise shape, n_loop = 1: (21, 6)
21
Pairwise shape, n_loop = 2: (102, 6)
['fraud-ip_country_us==False&num_distinct_txn_per_email_7day<=4', 'fraud-email_alpharatio<=0.33&num_distinct_txn_per_email_7day<=4']
Prec0.050955414012738856
Pairwise shape, n_loop = 2: (57, 6)
(87, 6)


/Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_optimisation/rule_optimisation/optimisation_functions.py:43: RuntimeWarning: invalid value encountered in long_scalars
  precision = np.nan_to_num(tps / tps_fps)
/Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_optimisation/rule_optimisation/optimisation_functions.py:43: RuntimeWarning: invalid value encountered in long_scalars
  precision = np.nan_to_num(tps / tps_fps)


,fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&num_distinct_txn_per_email_1day>=2,fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_1day>=2,fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&ip_country_us==False&num_distinct_txn_per_email_1day>=2,fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day<=4,fraud-email_kb_distance<=0.0&ip_country_us==False&num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_1day>=2,fraud-email_kb_distance<=0.0&num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_1day>=2,fraud-email_kb_distance<=0.0&num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day<=4,fraud-email_kb_distance<=0.0&ip_country_us==False&num_distinct_txn_per_email_1day>=2,fraud-email_kb_distance<=0.0&ip_country_us==False&num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day<=4,fraud-email_kb_distance<=0.0&num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day<=4,...,fraud-ip_country_us==False&num_distinct_txn_per_email_1day<=3,fraud-ip_country_us==False&num_distinct_txn_per_email_7day<=4,fraud-email_kb_distance>=0.94&num_distinct_txn_per_email_1day<=3,fraud-ip_country_us==False&num_distinct_txn_per_email_1day>=2,fraud-num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_1day>=2,fraud-num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day<=4,fraud-num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day>=7,fraud-email_kb_distance>=0.94&num_distinct_txn_per_email_7day>=7,fraud-ip_country_us==False&num_distinct_txn_per_email_7day>=7,fraud-num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_7day>=7
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,0,1,0,1,1,0,0,1,...,0,0,0,0,1,1,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,0,1,1,1,1
997,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0


In [17]:
# [rule_logic for rule_logic in rg.rule_descriptions['Logic'] if rule_logic not in rg_old.rule_descriptions.index]

In [18]:
rg_old.rule_descriptions

,Precision,Recall,nConditions,PercDataFlagged,Type,OptMetric
fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&num_distinct_txn_per_email_1day>=2,0.500000,0.20,3,0.008,Fraud,0.384615
fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_1day>=2,0.500000,0.20,4,0.008,Fraud,0.384615
fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&ip_country_us==False&num_distinct_txn_per_email_1day>=2,0.600000,0.15,4,0.005,Fraud,0.375000
fraud-email_alpharatio<=0.33&email_kb_distance<=0.0&num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day<=4,0.500000,0.15,4,0.006,Fraud,0.340909
fraud-email_kb_distance<=0.0&ip_country_us==False&num_distinct_txn_per_email_1day<=3&num_distinct_txn_per_email_1day>=2,0.333333,0.30,4,0.018,Fraud,0.326087
...,...,...,...,...,...,...
fraud-num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day<=4,0.044025,0.70,2,0.318,Fraud,0.000000
fraud-num_distinct_txn_per_email_1day>=2&num_distinct_txn_per_email_7day>=7,0.076923,0.10,2,0.026,Fraud,0.000000
fraud-email_kb_distance>=0.94&num_distinct_txn_per_email_7day>=7,0.071429,0.05,2,0.014,Fraud,0.000000
fraud-ip_country_us==False&num_distinct_txn_per_email_7day>=7,0.081081,0.15,2,0.037,Fraud,0.000000


In [43]:
def convert_to_old_syntax(x):
    return 'fraud-' + x.replace("(X['", "").replace("']", "").replace(")", "")

In [47]:
new_rules_old_syntax = rg.rule_descriptions['Logic'].apply(lambda x: convert_to_old_syntax(x)).values.tolist()

In [48]:
[r for r in new_rules_old_syntax if r not in rg_old.rule_descriptions.index]

[]

In [49]:
[r for r in rg_old.rule_descriptions.index if r not in new_rules_old_syntax]

['fraud-email_alpharatio<=0.33&ip_country_us==False&num_distinct_txn_per_email_7day<=4']

In [50]:
rg_old.rule_descriptions.loc["fraud-email_alpharatio<=0.33&ip_country_us==False&num_distinct_txn_per_email_7day<=4"]

Precision          0.0705882
Recall                   0.3
nConditions                3
PercDataFlagged        0.085
Type                   Fraud
OptMetric          0.0833333
Name: fraud-email_alpharatio<=0.33&ip_country_us==False&num_distinct_txn_per_email_7day<=4, dtype: object

In [ ]:
rg_old.rule_descriptions

In [83]:
rule_type = 'Fraud'
# sample_weight = weights
sample_weight = None

In [84]:
rule_descriptions, X_rules = argo_utils.generate_empty_data_structures()
columns_int, columns_cat, columns_float = argo_utils.return_columns_types(
    X)
columns_num = [
    col for col in columns_int if col not in columns_cat] + columns_float
if columns_num:
    rule_descriptions_num, X_rules_num = self._generate_numeric_one_condition_rules(
        X, y, columns_num, columns_int, rule_type, sample_weight)
    rule_descriptions, X_rules = argo_utils.combine_rule_dfs(
        rule_descriptions_num, X_rules_num, rule_descriptions, X_rules)
if columns_cat:
    rule_descriptions_cat, X_rules_cat = self._generate_categorical_one_condition_rules(
        X, y, columns_cat, rule_type, sample_weight)
    rule_descriptions, X_rules = argo_utils.combine_rule_dfs(
        rule_descriptions_cat, X_rules_cat, rule_descriptions, X_rules)
# self.rule_descriptions, X_rules = self._generate_n_order_pairwise_rules(
#     rule_descriptions, X_rules, y, rule_type, self.remove_corr_rules, sample_weight)

In [53]:
rules_t = list(map(convert_to_old_syntax, ["(X['email_alpharatio']<=0.33)&(X['ip_country_us']==False)", "(X['email_alpharatio']<=0.33)&(X['num_distinct_txn_per_email_7day']<=4)"]))

In [54]:
rg_old.rule_descriptions.loc[rules_t]

,Precision,Recall,nConditions,PercDataFlagged,Type,OptMetric
fraud-email_alpharatio<=0.33&ip_country_us==False,0.072727,0.4,2,0.110,Fraud,0.0
fraud-email_alpharatio<=0.33&num_distinct_txn_per_email_7day<=4,0.050955,0.4,2,0.157,Fraud,0.0


In [59]:
rg_old.rule_descriptions.loc[['fraud-ip_country_us==False&num_distinct_txn_per_email_7day<=4', 'fraud-email_alpharatio<=0.33&num_distinct_txn_per_email_7day<=4']]

,Precision,Recall,nConditions,PercDataFlagged,Type,OptMetric
fraud-ip_country_us==False&num_distinct_txn_per_email_7day<=4,0.033573,0.7,2,0.417,Fraud,0.0
fraud-email_alpharatio<=0.33&num_distinct_txn_per_email_7day<=4,0.050955,0.4,2,0.157,Fraud,0.0


In [60]:
dict.setdefault()

TypeError: descriptor 'setdefault' requires a 'dict' object but received a 'type'

In [88]:
from itertools import combinations

In [98]:
rule_logic = rule_descriptions['Logic'].values
rule_names = rule_descriptions.index.values
rules_logic_combinations = list(combinations(rule_logic, r=2))
rules_name_combinations = list(combinations(rule_names, r=2))
rule_combinations = list(zip(rules_logic_combinations, rules_name_combinations))

In [99]:
rule_combinations

[(("(X['ip_country_us']==False)", "(X['email_kb_distance']<=0.0)"),
  ('ARGORule18', 'ARGORule16')),
 (("(X['ip_country_us']==False)", "(X['email_alpharatio']<=0.33)"),
  ('ARGORule18', 'ARGORule17')),
 (("(X['ip_country_us']==False)", "(X['num_distinct_txn_per_email_1day']>=2)"),
  ('ARGORule18', 'ARGORule10')),
 (("(X['ip_country_us']==False)", "(X['num_distinct_txn_per_email_7day']>=7)"),
  ('ARGORule18', 'ARGORule11')),
 (("(X['ip_country_us']==False)", "(X['num_distinct_txn_per_email_1day']<=3)"),
  ('ARGORule18', 'ARGORule14')),
 (("(X['ip_country_us']==False)", "(X['num_distinct_txn_per_email_7day']<=4)"),
  ('ARGORule18', 'ARGORule15')),
 (("(X['ip_country_us']==False)", "(X['email_kb_distance']>=0.94)"),
  ('ARGORule18', 'ARGORule12')),
 (("(X['ip_country_us']==False)", "(X['email_alpharatio']>=0.5)"),
  ('ARGORule18', 'ARGORule13')),
 (("(X['email_kb_distance']<=0.0)", "(X['email_alpharatio']<=0.33)"),
  ('ARGORule16', 'ARGORule17')),
 (("(X['email_kb_distance']<=0.0)",
   "(

In [97]:
rules_name_combinations

[('ARGORule18', 'ARGORule16'),
 ('ARGORule18', 'ARGORule17'),
 ('ARGORule18', 'ARGORule10'),
 ('ARGORule18', 'ARGORule11'),
 ('ARGORule18', 'ARGORule14'),
 ('ARGORule18', 'ARGORule15'),
 ('ARGORule18', 'ARGORule12'),
 ('ARGORule18', 'ARGORule13'),
 ('ARGORule16', 'ARGORule17'),
 ('ARGORule16', 'ARGORule10'),
 ('ARGORule16', 'ARGORule11'),
 ('ARGORule16', 'ARGORule14'),
 ('ARGORule16', 'ARGORule15'),
 ('ARGORule16', 'ARGORule12'),
 ('ARGORule16', 'ARGORule13'),
 ('ARGORule17', 'ARGORule10'),
 ('ARGORule17', 'ARGORule11'),
 ('ARGORule17', 'ARGORule14'),
 ('ARGORule17', 'ARGORule15'),
 ('ARGORule17', 'ARGORule12'),
 ('ARGORule17', 'ARGORule13'),
 ('ARGORule10', 'ARGORule11'),
 ('ARGORule10', 'ARGORule14'),
 ('ARGORule10', 'ARGORule15'),
 ('ARGORule10', 'ARGORule12'),
 ('ARGORule10', 'ARGORule13'),
 ('ARGORule11', 'ARGORule14'),
 ('ARGORule11', 'ARGORule15'),
 ('ARGORule11', 'ARGORule12'),
 ('ARGORule11', 'ARGORule13'),
 ('ARGORule14', 'ARGORule15'),
 ('ARGORule14', 'ARGORule12'),
 ('ARGOR

In [68]:
rule_descriptions_num

,Logic,Precision,Recall,nConditions,PercDataFlagged,OptMetric
Rule,,,,,,
ARGORule6,(X['email_kb_distance']<=0.0),0.090909,0.35,1,0.077,0.106707
ARGORule7,(X['email_alpharatio']<=0.33),0.050505,0.50,1,0.198,0.061576
ARGORule0,(X['num_distinct_txn_per_email_1day']>=2),0.043702,0.85,1,0.389,0.053934
ARGORule1,(X['num_distinct_txn_per_email_7day']>=7),0.042254,0.15,1,0.071,0.049342
ARGORule4,(X['num_distinct_txn_per_email_1day']<=3),0.022173,1.00,1,0.902,0.027563
ARGORule5,(X['num_distinct_txn_per_email_7day']<=4),0.019975,0.80,1,0.801,0.024814
ARGORule2,(X['email_kb_distance']>=0.94),0.020101,0.20,1,0.199,0.024510
ARGORule3,(X['email_alpharatio']>=0.5),0.004024,0.10,1,0.497,0.004980


In [69]:
rule_descriptions_cat

,Logic,Precision,Recall,nConditions,PercDataFlagged,OptMetric
Rule,,,,,,
ARGORule8,(X['ip_country_us']==False),0.032136,0.85,1,0.529,0.039794


In [24]:
import argo_utils.argo_utils as argo_utils
import math

In [25]:
rg = RuleGeneratorOpt(opt_func=f.fit, n_total_conditions=4, num_rules_keep=50, n_points= 10, ratio_window= 2, 
                      remove_corr_rules= False)

In [29]:
rule_descriptions = pd.DataFrame(
    {
        'Rule': ['A', 'B', 'C'],
        'Precision': [1, 0.5, 0]
    }
)
rule_descriptions.set_index('Rule', inplace=True)
pairwise_descriptions = pd.DataFrame(
    {
        'Rule': ['A&B', 'B&C', 'A&C'],
        'Precision': [1, 0.75, 0]
    }
)
pairwise_descriptions.set_index('Rule', inplace=True)
X_rules_pairwise = pd.DataFrame({
    'A&B': range(0, 1000),
    'B&C': range(0, 1000),
    'A&C': range(0, 1000),
})
pairwise_to_orig_lookup = {
    'A&B': ['A', 'B'],
    'A&C': ['A', 'C'],
    'B&C': ['B', 'C'],
}

In [30]:
args = pairwise_descriptions, X_rules_pairwise, pairwise_to_orig_lookup, rule_descriptions

In [31]:
pairwise_descriptions, X_rules_pairwise = rg._drop_unnecessary_pairwise_rules(*args)

In [32]:
pairwise_descriptions

,Precision
Rule,
B&C,0.75


In [122]:
import string
from sklearn.metrics import precision_score, recall_score

In [113]:
np.random.seed(0)
y_preds = pd.DataFrame(np.random.randint(0, 2, size=(1000, 10)), columns=[
                       i for i in string.ascii_letters[:10]])
y_true = pd.Series(np.random.randint(0, 2, 1000))
sample_weight = y_true * 10

In [116]:
import databricks.koalas as ks

In [117]:
y_preds_ks = ks.from_pandas(y_preds)
y_true_ks = ks.from_pandas(y_true)

In [164]:
sample_weight_ks = ks.from_pandas(sample_weight)

In [166]:
# def return_binary_pred_perf_of_set_pandas(y_true, y_preds,
#                                          sample_weight=None, opt_func=None):
#     total_num_flagged = y_preds_ks.sum().to_pandas() 
#     total_perc_flagged = y_preds_ks.mean().to_pandas()
#     new_col_names = []
#     if sample_weight is not None:
#         y_preds_and_true = ks.concat([y_preds_ks, y_true_ks.rename('y_true'), sample_weight.rename('sample_weight')], axis=1)
#     else:        
#         y_preds_and_true = ks.concat([y_preds_ks, y_true_ks.rename('y_true')], axis=1)
#     for col in y_preds_and_true:
#         if col == 'y_true' or col == 'sample_weight':
#             continue
#         new_col_name = f'{col}_equals_y_true'
#         if sample_weight is not None:
#             y_preds_and_true[new_col_name] = y_preds_and_true[col] * y_preds_and_true['y_true'] * y_preds_and_true['sample_weight']
#         else:
#             y_preds_and_true[new_col_name] = y_preds_and_true[col] * y_preds_and_true['y_true']
#         new_col_names.append(new_col_name)
#     preds_equal_true = y_preds_and_true[new_col_names].sum().to_pandas()
#     precisions = pd.Series(preds_equal_true.values/total_num_flagged.values, y_preds.columns, name='Precision')
#     return precisions

In [167]:
return_binary_pred_perf_of_set_pandas(y_true_ks, y_preds_ks, sample_weight_ks)

a    5.097276
b    5.186640
c    5.134615
d    5.335968
e    5.328032
f    5.454545
g    5.072165
h    5.162524
i    5.075758
j    5.059761
Name: Precision, dtype: float64

In [52]:
from rule_application.argo_rule_applier import ArgoRuleApplier

In [53]:
ara = ArgoRuleApplier(rule_strings={'Rule1:': "(X['email_kb_distance']<=0.0)"})

In [54]:
ara.apply(X, y)

,Rule1:
0,0
1,0
2,0
3,0
4,0
...,...
995,1
996,0
997,0
998,0


In [55]:
ara.rule_descriptions

,Logic,Precision,Recall,nConditions,PercDataFlagged,OptMetric
Rule,,,,,,
Rule1:,(X['email_kb_distance']<=0.0),0.090909,0.35,1,0.077,None


In [137]:
np.random.seed(0)
X = pd.DataFrame(
    {
        'A': np.random.randint(0, 100, 1000000),
        'B': np.random.randint(0, 100, 1000000),
        'C': ['a', 'b&'] * 500000
    }
)

In [114]:
rule1 = (X['A']>2).astype(int)
rule2 = (X['B']>3).astype(int)

In [116]:
%timeit rule1*rule2

2.23 ms ± 373 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [117]:
%timeit eval("(X['A']>2).astype(int)&(X['B']>3).astype(int)")

8.31 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [138]:
%timeit X['C'].apply(argo_utils.count_rule_conditions)

389 ms ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [142]:
X['C'].values

array(['a', 'b&', 'a', ..., 'b&', 'a', 'b&'], dtype=object)

In [140]:
%timeit np.apply_along_axis(argo_utils.count_rule_conditions, 0, X['C'].values)

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [182]:
random.seed(0)
np.random.seed(0)
y = pd.Series(data=[0]*5 + [1]*5, index=list(range(0, 10)))
X = pd.DataFrame(data={
    'A': [5,0,5,0,5,3,4,0,0,0],
    'B': [0,1,0,1,0,1,0.6,0.7,0,0],
    'C_US': [1,1,1,1,1,1,0,0,1,1]
},
    index=list(range(0, 10))
)
columns_int = [
    'A']
columns_cat = ['C']
columns_num = ['A', 'B']
weights = y.apply(lambda x: 1000 if x == 1 else 1)

In [183]:
X

,A,B,C_US
0,5,0.0,1
1,0,1.0,1
2,5,0.0,1
3,0,1.0,1
4,5,0.0,1
5,3,1.0,1
6,4,0.6,0
7,0,0.7,0
8,0,0.0,1
9,0,0.0,1


In [56]:
rules = {'ARGORule0': "(X['A']>=3)", 'ARGORule1':"(X['B']>=0.5)", 'ARGORule2':"(X['C_US']==True)"}

In [57]:
from rule_application.argo_rule_applier import ArgoRuleApplier
from rule_optimisation.optimisation_functions import FScore

In [58]:
f0dot5 = FScore(0.5)

In [59]:
ar = ArgoRuleApplier(rules, f0dot5.fit)

In [60]:
X_rules = ar.apply(X, y)

In [61]:
X_rules

,ARGORule1,ARGORule2,ARGORule0
0,0,1,1
1,1,1,0
2,0,1,1
3,1,1,0
4,0,1,1
5,1,1,1
6,1,0,1
7,1,0,0
8,0,1,0
9,0,1,0


In [62]:
ar.rule_descriptions

,Logic,Precision,Recall,nConditions,PercDataFlagged,OptMetric
Rule,,,,,,
ARGORule1,(X['B']>=0.5),0.600,0.6,1,0.5,0.600000
ARGORule2,(X['C_US']==True),0.375,0.6,1,0.8,0.405405
ARGORule0,(X['A']>=3),0.400,0.4,1,0.5,0.400000


In [191]:
# test = pd.DataFrame(
#     np.array([["(X['B']>=0.5)", 0.9993337774816788, 0.6, 1, 0.5,
#         0.8819379115710255],
#        ["(X['C_US']==True)", 0.9983361064891847, 0.6, 1, 0.8,
#         0.8813160987074031],
#        ["(X['A']>=3)", 0.9985022466300549, 0.4, 1, 0.5,
#         0.7685213648939442]]),
#     columns = ['Logic', 'Precision', 'Recall', 'nConditions', 'PercDataFlagged', 'OptMetric'],
#     index=['ARGORule1', 'ARGORule2', 'ARGORule0'],
# )

In [192]:
# test = pd.DataFrame(
#             np.array([[0, 1, 1],
#                       [1, 1, 0],
#                       [0, 1, 1],
#                       [1, 1, 0],
#                       [0, 1, 1],
#                       [1, 1, 1],
#                       [1, 0, 1],
#                       [1, 0, 0],
#                       [0, 1, 0],
#                       [0, 1, 0]], dtype=np.int),
#             columns=['ARGORule1', 'ARGORule2', 'ARGORule0'],
#         )

In [155]:
# test.astype({'Logic': object, 'Precision': float, 'Recall':float, 'nConditions':int, 'PercDataFlagged':float, 'OptMetric':float})

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
from rule_application.argo_rule_applier import ArgoRuleApplier
from rule_optimisation.optimisation_functions import FScore
from rule_generation.rule_generator_opt import RuleGeneratorOpt
import argo_utils.argo_utils as argo_utils
import numpy as np
import pandas as pd
import random
import math

In [133]:
random.seed(0)
np.random.seed(0)
y = pd.Series(data=[0]*5 + [1]*5, index=list(range(0, 10)))
X = pd.DataFrame(data={
    'A': [5, 0, 5, 0, 5, 3, 4, 0, 0, 0],
    'B': [0, 1, 0, 1, 0, 1, 0.6, 0.7, 0, 0],
    'C_US': [1, 1, 1, 1, 1, 1, 0, 0, 1, 1]
},
    index=list(range(0, 10))
)
columns_int = ['A']
columns_cat = ['C']
columns_num = ['A', 'B']
weights = y.apply(lambda x: 1000 if x == 1 else 1)

In [134]:
f0dot5 = FScore(0.5)

In [135]:
params = {
    'opt_func': f0dot5.fit,
    'n_total_conditions': 4,
    'num_rules_keep': 50,
    'n_points': 10,
    'ratio_window': 2,
    'remove_corr_rules': False
}
rg = RuleGeneratorOpt(**params)

In [136]:
weight_is_none = True

In [137]:
if weight_is_none:
    rule_descriptions = pd.DataFrame(
        np.array([["(X['B']>=0.5)", 0.6, 0.6, 1, 0.5, 0.6],
                  ["(X['C_US']==True)", 0.375, 0.6,
                   1, 0.8, 0.4054054054054054],
                  ["(X['A']>=3)", 0.4, 0.4, 1, 0.5, 0.4000000000000001]]),
        columns=['Logic', 'Precision', 'Recall',
                 'nConditions', 'PercDataFlagged', 'OptMetric'],
        index=['ARGORule1', 'ARGORule2', 'ARGORule0'],
    )
    rule_descriptions = rule_descriptions.astype({'Logic': object, 'Precision': float, 'Recall': float,
                                                  'nConditions': int, 'PercDataFlagged': float, 'OptMetric': float})
    rule_descriptions.index.name = 'Rule'
    weights = None
else:
    rule_descriptions = pd.DataFrame(
        np.array([["(X['B']>=0.5)", 0.9993337774816788, 0.6, 1, 0.5,
                   0.8819379115710255],
                  ["(X['C_US']==True)", 0.9983361064891847, 0.6, 1, 0.8,
                   0.8813160987074031],
                  ["(X['A']>=3)", 0.9985022466300549, 0.4, 1, 0.5,
                   0.7685213648939442]]),
        columns=['Logic', 'Precision', 'Recall',
                 'nConditions', 'PercDataFlagged', 'OptMetric'],
        index=['ARGORule1', 'ARGORule2', 'ARGORule0'],
    )
    rule_descriptions = rule_descriptions.astype({'Logic': object, 'Precision': float, 'Recall': float,
                                                  'nConditions': int, 'PercDataFlagged': float, 'OptMetric': float})
    rule_descriptions.index.name = 'Rule'
    weights = weights
X_rules = pd.DataFrame(
    np.array([[0, 1, 1],
              [1, 1, 0],
              [0, 1, 1],
              [1, 1, 0],
              [0, 1, 1],
              [1, 1, 1],
              [1, 0, 1],
              [1, 0, 0],
              [0, 1, 0],
              [0, 1, 0]], dtype=np.int),
    columns=['ARGORule1', 'ARGORule2', 'ARGORule0'],
)
rule_combinations = [(('ARGORule1', 'ARGORule2'), ("(X['B']>=0.5)", "(X['C_US']==True)")),
                     (('ARGORule1', 'ARGORule0'),
                      ("(X['B']>=0.5)", "(X['A']>=3)")),
                     (('ARGORule2', 'ARGORule0'), ("(X['C_US']==True)", "(X['A']>=3)"))]

In [138]:
self = rg
self.rule_name_counter = 3
sample_weight = weights
remove_corr_rules = True

In [139]:
n_loops = int(
    math.log(2 ** math.ceil(math.log(self.n_total_conditions, 2)), 2))
for n_loop in range(1, n_loops + 1):
    rules_combinations = self._get_rule_combinations_for_loop(
        rule_descriptions, n_loop, self.num_rules_keep)
    if len(rules_combinations) == 0:
        break
    rule_descriptions_pairwise, X_rules_pairwise, pairwise_components = self._generate_pairwise_rules(
        rule_descriptions, X_rules, y, rules_combinations, sample_weight)
    rule_descriptions_pairwise, X_rules_pairwise = self._drop_unnecessary_pairwise_rules(
        rule_descriptions_pairwise, X_rules_pairwise,  pairwise_components, rule_descriptions)
    X_rules = pd.concat(
        [X_rules, X_rules_pairwise], axis=1)
    rule_descriptions = pd.concat(
        [rule_descriptions, rule_descriptions_pairwise], axis=0)
    if remove_corr_rules:
        print('yup')
        cs = CosineSimilarity()
        rcr = AgglomerativeClusteringFeatureReduction(
            threshold=0.75, columns_performance=rule_descriptions['OptMetric'], strategy='bottom_up', similarity_function=cs.fit)
        X_rules = rcr.fit_transform(X_rules)
        rule_descriptions = rule_descriptions.loc[X_rules.columns]
rule_descriptions = rule_descriptions[rule_descriptions['nConditions']
                                      <= self.n_total_conditions]
X_rules = X_rules[rule_descriptions.index.tolist()]
rule_descriptions, X_rules = argo_utils.sort_rule_dfs_by_opt_metric(
    rule_descriptions, X_rules)


ARGORule3
ARGORule4
ARGORule5
yup


In [140]:
rule_descriptions

,Logic,Precision,Recall,nConditions,PercDataFlagged,OptMetric
ARGORule4,(X['A']>=3)&(X['B']>=0.5),1.000,0.4,2,0.2,0.769231
ARGORule1,(X['B']>=0.5),0.600,0.6,1,0.5,0.600000
ARGORule2,(X['C_US']==True),0.375,0.6,1,0.8,0.405405
ARGORule0,(X['A']>=3),0.400,0.4,1,0.5,0.400000


In [131]:
# rg._generate_n_order_pairwise_rules(rule_descriptions=rule_descriptions, X_rules=X_rules, y=y, remove_corr_rules=remove_corr_rules, sample_weight=weights)

In [116]:
from correlation_reduction.correlation_reduction_methods import AgglomerativeClusteringFeatureReduction
from correlation_reduction.similarity_functions import CosineSimilarity

In [120]:
cs = CosineSimilarity()
rcr = AgglomerativeClusteringFeatureReduction(
    threshold=0.75, columns_performance=rule_descriptions['OptMetric'], strategy='bottom_up', similarity_function=cs.fit)
X_rules = rcr.fit_transform(X_rules)

In [121]:
X_rules

,ARGORule1,ARGORule2
0,0,1
1,1,1
2,0,1
3,1,1
4,0,1
5,1,1
6,1,0
7,1,0
8,0,1
9,0,1


In [85]:
X_rules_pairwise_arr = X_rules[['ARGORule1']].values * X_rules[['ARGORule1']].values

In [86]:
X_rules_pairwise_arr

array([[0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0]])

In [87]:
pd.DataFrame(X_rules_pairwise_arr, columns=pairwise_names, index=X_rules.index)

NameError: name 'pairwise_names' is not defined